###**Q1**.  Use Lucidchart to create an Entity Relationship Diagram (ERD) for the following tables representing a customer order tracking system:

Tables and fields:
1. product:
    - product_id: INTEGER (Primary key)
    - name: TEXT not null
    - price: REAL


2. customer:
    - customer_id: INTEGER (Primary key)
    - name: TEXT not null
    - email: TEXT not null


3. purchase_order:
    - order_id: INTEGER (Primary key)
    - customer_id: INTEGER (Foreign key)
    - date: TEXT not null ("YYYY-MM-DD")


4. order_item:
    - order_id: INTEGER (Foreign key)
    - product_id: INTEGER (Foreign key)
    - quantity: INTEGER
    


Export the ERD as PDF and submit it in Canvas.

###**Q2**. Create SQLite tables and load data
1. Here are the csv files for the data for the four tables:
   - product: https://raw.githubusercontent.com/csbfx/cs133/main/product.csv
   - customer: https://raw.githubusercontent.com/csbfx/cs133/main/customer.csv
   - order_item: https://raw.githubusercontent.com/csbfx/cs133/main/order_item.csv
   - purchase_order: https://raw.githubusercontent.com/csbfx/cs133/main/purchase_order.csv
2. In this notebook, create the database and save it in a file called `store.db`, and create the four tables as described above.
3. Load the data in the csv files into the corresponding table.
4. Commit so that the data loaded to the tables to officially written to the tables.
5. Execute a query SELECT * from each table to make sure the data are properly loaded.
6. Execute a query using SELECT statement that queries with JOIN tables to find the purchase date, the products and quantities that a particular customer has purchased.

In [3]:
import sqlite3
from pathlib import Path
from google.colab import drive
drive.mount('/content/drive')
gdrive='/content/drive/MyDrive/CS133/HO'
Path(gdrive+'store.db').touch()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# 2.2 Create the database and save it in a file called store.db, and create the four tables as described above.
conn = sqlite3.connect('store.db')
c = conn.cursor()

c.execute('''
CREATE TABLE IF NOT EXISTS product (
    product_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    price REAL
)
''')

c.execute('''
CREATE TABLE IF NOT EXISTS customer (
    customer_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    email TEXT NOT NULL
)
''')

c.execute('''
CREATE TABLE IF NOT EXISTS purchase_order (
    order_id INTEGER PRIMARY KEY,
    customer_id INTEGER,
    date TEXT NOT NULL,
    FOREIGN KEY (customer_id) REFERENCES customer(customer_id)
)
''')

c.execute('''
CREATE TABLE IF NOT EXISTS order_item (
    order_id INTEGER,
    product_id INTEGER,
    quantity INTEGER,
    FOREIGN KEY (order_id) REFERENCES purchase_order(order_id),
    FOREIGN KEY (product_id) REFERENCES product(product_id)
)
''')

conn.commit()

In [7]:
# 2.3 Load the data in the csv files into the corresponding table.
import pandas as pd

data = pd.read_csv('https://raw.githubusercontent.com/csbfx/cs133/main/product.csv')
data.to_sql('product', conn, if_exists='replace', index=False)

data = pd.read_csv('https://raw.githubusercontent.com/csbfx/cs133/main/customer.csv')
data.to_sql('customer', conn, if_exists='replace', index=False)

data = pd.read_csv('https://raw.githubusercontent.com/csbfx/cs133/main/order_item.csv')
data.to_sql('order_item', conn, if_exists='replace', index=False)

data = pd.read_csv('https://raw.githubusercontent.com/csbfx/cs133/main/purchase_order.csv')
data.to_sql('purchase_order', conn, if_exists='replace', index=False)

3

In [8]:
# 2.4 Commit so that the data loaded to the tables to officially written to the tables.
conn.commit()

In [12]:
# 2.5 Execute a query SELECT * from each table to make sure the data are properly loaded.
c.execute("SELECT * FROM product")
print(c.fetchall())
c.execute("SELECT * FROM customer")
print(c.fetchall())
c.execute("SELECT * FROM order_item")
print(c.fetchall())
c.execute("SELECT * FROM purchase_order")
print(c.fetchall())

[(0, 'bicycle', 400), (1, 'helmet', 45), (2, 'gloves', 23), (3, 'chain', 48)]
[(0, 'Wendy Lee', 'wlee@bike.com'), (1, 'Jason Brown', 'jb@speed.com'), (2, 'Harry Potter', 'hp@hogwarts.edu'), (3, 'Godric Gryffindor', 'gg@hogwards.edu')]
[(0, 0, 1), (0, 1, 1), (1, 2, 2), (1, 3, 1)]
[(0, 0, '2020-10-19'), (1, 0, '2020-10-20'), (2, 1, '2020-10-20')]


In [16]:
# 2.6 Execute a query using SELECT statement that queries with JOIN tables to find the purchase date,
#    the products and quantities that a particular customer has purchased.
query = """
SELECT
    c.name AS Customer_Name,
    po.date AS Purchase_Date,
    p.name AS Product,
    oi.quantity AS Quantity
FROM
    customer c
JOIN
    purchase_order po ON c.customer_id = po.customer_id
JOIN
    order_item oi ON po.order_id = oi.order_id
JOIN
    product p ON oi.product_id = p.product_id
"""

c.execute(query)
print(c.fetchall())
# i would expect James Brown here, but the order_item for his order does not exist.

[('Wendy Lee', '2020-10-19', 'bicycle', 1), ('Wendy Lee', '2020-10-19', 'helmet', 1), ('Wendy Lee', '2020-10-20', 'gloves', 2), ('Wendy Lee', '2020-10-20', 'chain', 1)]
